# Home Mortgage Disclosure Act Final Project

In [2]:
import warnings
import pandas as pd
warnings.filterwarnings("ignore", category=pd.errors.SettingWithCopyWarning)


### Step 1. Dataset Selection

In [3]:
file = "state_GA.csv" 
data = pd.read_csv(file)

data_reduced = data[['lei', 'derived_ethnicity', 'derived_race', 'derived_sex', 'loan_amount', 'debt_to_income_ratio', 'loan_purpose', 'loan_term', 'action_taken', 'denial_reason-1', 'denial_reason-2', 
                     'income', 'applicant_age', 'total_loan_costs', 'interest_rate', 'applicant_race-1', 'applicant_race-2', 'applicant_sex','co-applicant_sex']]

/var/folders/mk/0rr9xxs16xg_99dy8_m3zqrw0000gn/T/ipykernel_16310/2768295705.py:2: DtypeWarning: Columns (22,23,24,26,27,28,29,30,31,32,33,38,43,44) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file)


In [4]:
## Only Need to run once
data_reduced.to_csv("state_GA_reduced.csv", index=False)
hmda_data = pd.read_csv("state_GA_reduced.csv")

In [5]:
sex_map = {
    1: "Male",
    2: "Female",
    6: "Applicant selected both male and female"
}

race_map = {
    1: "American Indian or Alaska Native",
    2: "Asian",
    21: "Asian Indian",
    22: "Chinese",
    23: "Filipino",
    24: "Japanese",
    25: "Korean",
    26: "Vietnamese",
    27: "Other Asian",
    3: "Black or African American",
    4: "Native Hawaiian or Other Pacific Islander",
    41: "Native Hawaiian",
    42: "Guamanian or Chamorro",
    43: "Samoan",
    44: "Other Pacific Islander",
    5: "White"
}

data_reduced['applicant_race-1'] = pd.to_numeric(data_reduced['applicant_race-1'], errors='coerce')
data_reduced['applicant_race-2'] = pd.to_numeric(data_reduced['applicant_race-2'], errors='coerce')
data_reduced['applicant_sex'] = pd.to_numeric(data_reduced['applicant_sex'], errors='coerce')
data_reduced['co-applicant_sex'] = pd.to_numeric(data_reduced['co-applicant_sex'], errors='coerce')

filtered = data_reduced[
    data_reduced['applicant_race-1'].isin(race_map.keys()) &
    (
        data_reduced['applicant_race-2'].isna() |
        data_reduced['applicant_race-2'].isin(race_map.keys())
    ) &
    data_reduced['applicant_sex'].isin(sex_map.keys()) &
    (
        data_reduced['co-applicant_sex'].isna() |
        data_reduced['co-applicant_sex'].isin(sex_map.keys())
    )

]

filtered['race_1_str'] = filtered['applicant_race-1'].map(race_map)
filtered['race_2_str'] = filtered['applicant_race-2'].map(race_map)
def combine_races_str(row):
    if pd.isna(row['race_2_str']) or row['race_2_str'] == "":
        return row['race_1_str']
    return f"{row['race_1_str']}, {row['race_2_str']}"
def combine_sexs_str(row):
    if pd.isna(row['co-applicant_sex']) or row['co-applicant_sex'] == "":
        return row['applicant_sex']
    return f"{row['applicant_sex']}, {row['co-applicant_sex']}"

filtered['derived_race_new'] = filtered.apply(combine_races_str, axis=1)

filtered['applicant_sex'] = filtered['applicant_sex'].map(sex_map)
filtered['co-applicant_sex'] = filtered['co-applicant_sex'].map(sex_map)

filtered['derived_sex_new'] = filtered.apply(combine_sexs_str, axis=1)

filtered = filtered[filtered['action_taken'] != 6]
filtered = filtered[filtered['interest_rate'] != 'Exempt']


filtered['favorable_action_taken'] = filtered['action_taken'].apply(
    lambda x: 1 if x in [1, 2, 8] else (0 if x in [3, 4, 5, 7] else pd.NA)
)
filtered['interest_rate'].unique()
filtered['favorable_interest_rate'] = filtered['interest_rate'].apply(
    lambda x: 1 if float(x) <= 7.5 else 0)

unique_races = sorted(filtered['derived_race_new'].unique())
unique_sexes = sorted(filtered['derived_sex_new'].unique())

# New encoding dictionaries
final_race_encoding = {race: i for i, race in enumerate(unique_races)}
final_sex_encoding = {sex: i for i, sex in enumerate(unique_sexes)}

filtered['derived_race_encoded'] = filtered['derived_race_new'].map(final_race_encoding)
filtered['derived_sex_encoded'] = filtered['derived_sex_new'].map(final_sex_encoding)


filtered.to_csv("state_GA_reduced_encoded.csv", index=False)

In [9]:
import matplotlib.pyplot as plt
import seaborn as sns
import os
protected_classes = {
    "derived_race_new": "Race",
    "derived_sex_new": "Sex"
}
dependent_variables = {
    "favorable_action_taken": "Favorable Action Taken",
    "favorable_interest_rate": "Favorable Interest Rate"
}

os.makedirs("charts", exist_ok=True)

tables = []
for protected_var, protected_label in protected_classes.items():
    for dependent_var, dependent_label in dependent_variables.items():
        freq_table = pd.crosstab(filtered[protected_var], filtered[dependent_var], margins=True, dropna=False)
        freq_table = freq_table.loc[freq_table["All"].sort_values(ascending=False).index]
        freq_table.columns.name = dependent_label
        freq_table.index.name = protected_label
        tables.append((protected_var, dependent_var, freq_table))

        plt.figure(figsize=(10, 6))
        sns.countplot(data=filtered, x=protected_var, hue=dependent_var)
        plt.title(f"{protected_label} vs {dependent_label}")
        plt.xlabel(protected_label)
        plt.ylabel("Count")
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        plt.savefig(f"charts/{protected_var}_vs_{dependent_var}.png")
        plt.close()

for protected_var, dependent_var, table in tables:
    print(f"\n===== Frequency Table: {protected_var} vs {dependent_var} =====\n")
    display(table)
for protected_var, protected_label in protected_classes.items():
    print(f"\n===== Top 10 Raw Frequencies for: {protected_label} ({protected_var}) =====\n")
    print(filtered[protected_var].value_counts(dropna=False).sort_values(ascending=False).head(10))



===== Frequency Table: derived_race_new vs favorable_action_taken =====



Favorable Action Taken,0,1,All
Race,,,
All,38759,67436,106195
White,25211,50230,75441
Black or African American,10085,11103,21188
"Asian, Asian Indian",816,1665,2481
Asian,446,778,1224
...,...,...,...
"Japanese, White",0,1,1
"Korean, Asian",0,1,1
"Native Hawaiian or Other Pacific Islander, Asian Indian",0,1,1



===== Frequency Table: derived_race_new vs favorable_interest_rate =====



Favorable Interest Rate,0,1,All
Race,,,
All,55453,50742,106195
White,38515,36926,75441
Black or African American,12537,8651,21188
"Asian, Asian Indian",1041,1440,2481
Asian,613,611,1224
...,...,...,...
"Japanese, White",0,1,1
"Korean, Asian",0,1,1
"Native Hawaiian or Other Pacific Islander, Asian Indian",0,1,1



===== Frequency Table: derived_sex_new vs favorable_action_taken =====



Favorable Action Taken,0,1,All
Sex,,,
All,38759,67436,106195
"Male, Female",22844,41938,64782
"Female, Male",12134,19722,31856
"Female, Female",2125,3028,5153
"Male, Male",1513,2550,4063
"Applicant selected both male and female, Female",63,89,152
"Applicant selected both male and female, Male",43,62,105
"Male, Applicant selected both male and female",19,23,42
"Female, Applicant selected both male and female",13,16,29



===== Frequency Table: derived_sex_new vs favorable_interest_rate =====



Favorable Interest Rate,0,1,All
Sex,,,
All,55453,50742,106195
"Male, Female",33599,31183,64782
"Female, Male",16990,14866,31856
"Female, Female",2687,2466,5153
"Male, Male",2002,2061,4063
"Applicant selected both male and female, Female",79,73,152
"Applicant selected both male and female, Male",51,54,105
"Male, Applicant selected both male and female",23,19,42
"Female, Applicant selected both male and female",15,14,29



===== Top 10 Raw Frequencies for: Race (derived_race_new) =====

derived_race_new
White                               75441
Black or African American           21188
Asian, Asian Indian                  2481
Asian                                1224
Asian Indian                          829
Asian, Vietnamese                     705
Asian, Chinese                        599
Asian, Korean                         599
Asian, Other Asian                    493
Black or African American, White      385
Name: count, dtype: int64

===== Top 10 Raw Frequencies for: Sex (derived_sex_new) =====

derived_sex_new
Male, Female                                                                        64782
Female, Male                                                                        31856
Female, Female                                                                       5153
Male, Male                                                                           4063
Applicant selected both male an

In [12]:
# Get top 10 derived races with their encoded values (excluding counts)
top_10_races = (
    filtered[['derived_race_new', 'derived_race_encoded']]
    .drop_duplicates()
    .set_index('derived_race_new')
    .loc[filtered['derived_race_new'].value_counts().head(10).index]
    .reset_index()
    .rename(columns={
        'derived_race_new': 'Derived Race',
        'derived_race_encoded': 'Encoded Value'
    })
)

print(top_10_races.to_latex(index=False, caption="Top 10 Derived Race Strings with Encoded Values", label="tab:top_race_encoded"))

# Get top 10 derived sexes with their encoded values (excluding counts)
top_10_sexes = (
    filtered[['derived_sex_new', 'derived_sex_encoded']]
    .drop_duplicates()
    .set_index('derived_sex_new')
    .loc[filtered['derived_sex_new'].value_counts().head(10).index]
    .reset_index()
    .rename(columns={
        'derived_sex_new': 'Derived Sex',
        'derived_sex_encoded': 'Encoded Value'
    })
)

print(top_10_sexes.to_latex(index=False, caption="Top 10 Derived Sex Strings with Encoded Values", label="tab:top_sex_encoded"))


\begin{table}
\caption{Top 10 Derived Race Strings with Encoded Values}
\label{tab:top_race_encoded}
\begin{tabular}{lr}
\toprule
Derived Race & Encoded Value \\
\midrule
White & 75 \\
Black or African American & 26 \\
Asian, Asian Indian & 15 \\
Asian & 8 \\
Asian Indian & 9 \\
Asian, Vietnamese & 24 \\
Asian, Chinese & 17 \\
Asian, Korean & 20 \\
Asian, Other Asian & 22 \\
Black or African American, White & 38 \\
\bottomrule
\end{tabular}
\end{table}

\begin{table}
\caption{Top 10 Derived Sex Strings with Encoded Values}
\label{tab:top_sex_encoded}
\begin{tabular}{lr}
\toprule
Derived Sex & Encoded Value \\
\midrule
Male, Female & 7 \\
Female, Male & 5 \\
Female, Female & 4 \\
Male, Male & 8 \\
Applicant selected both male and female, Female & 1 \\
Applicant selected both male and female, Male & 2 \\
Male, Applicant selected both male and female & 6 \\
Female, Applicant selected both male and female & 3 \\
Applicant selected both male and female, Applicant selected both male and fema

In [13]:
# Ensure only top 10 race groups are used
top_10_races = filtered['derived_race_new'].value_counts().head(10).index
filtered_race = filtered[filtered['derived_race_new'].isin(top_10_races)]

tables = []

for protected_var, protected_label in {
    'derived_race_new': 'Race',
    'derived_sex_new': 'Sex'
}.items():
    subset = filtered_race if protected_var == 'derived_race_new' else filtered

    for dependent_var, dependent_label in dependent_variables.items():
        freq_table = pd.crosstab(subset[protected_var], subset[dependent_var], dropna=False)
        freq_table.columns.name = dependent_label
        freq_table.index.name = protected_label
        tables.append((protected_var, dependent_var, freq_table))

        plt.figure(figsize=(10, 6))
        sns.countplot(data=subset, x=protected_var, hue=dependent_var,
                      order=subset[protected_var].value_counts().head(10).index if protected_var == 'derived_race_new' else None)
        plt.title(f"{protected_label} vs {dependent_label}")
        plt.xlabel(protected_label)
        plt.ylabel("Count")
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        plt.savefig(f"charts/{protected_var}_vs_{dependent_var}.png")
        plt.close()

# Display all frequency tables
for protected_var, dependent_var, table in tables:
    print(f"\n===== Frequency Table: {protected_var} vs {dependent_var} =====\n")
    print(table.to_latex(index=True, caption=f"{protected_var} vs {dependent_var}", label=f"tab:{protected_var}_vs_{dependent_var}"))



===== Frequency Table: derived_race_new vs favorable_action_taken =====

\begin{table}
\caption{derived_race_new vs favorable_action_taken}
\label{tab:derived_race_new_vs_favorable_action_taken}
\begin{tabular}{lrr}
\toprule
Favorable Action Taken & 0 & 1 \\
Race &  &  \\
\midrule
Asian & 446 & 778 \\
Asian Indian & 353 & 476 \\
Asian, Asian Indian & 816 & 1665 \\
Asian, Chinese & 162 & 437 \\
Asian, Korean & 180 & 419 \\
Asian, Other Asian & 173 & 320 \\
Asian, Vietnamese & 221 & 484 \\
Black or African American & 10085 & 11103 \\
Black or African American, White & 180 & 205 \\
White & 25211 & 50230 \\
\bottomrule
\end{tabular}
\end{table}


===== Frequency Table: derived_race_new vs favorable_interest_rate =====

\begin{table}
\caption{derived_race_new vs favorable_interest_rate}
\label{tab:derived_race_new_vs_favorable_interest_rate}
\begin{tabular}{lrr}
\toprule
Favorable Interest Rate & 0 & 1 \\
Race &  &  \\
\midrule
Asian & 613 & 611 \\
Asian Indian & 417 & 412 \\
Asian, Asian 

In [23]:
%pip install plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots
bar_colors = ['#a6bddb', '#fcae91']
pattern_shapes = ['\\', '/']

def abbreviate_labels(labels):
    abbr = {label: f"{label}" if 'both' not in label else ', '.join([i if 'both' not in i else 'both' for i in label.split(',')]) for i, label in enumerate(labels)}
    return abbr, list(abbr.values()), abbr.items()

for protected_var, protected_label in {
    'derived_race_new': 'Race',
    'derived_sex_new': 'Sex'
}.items():
    subset = filtered_race if protected_var == 'derived_race_new' else filtered
    category_order = (
        subset[protected_var]
        .value_counts()
        .head(10 if protected_var == 'derived_race_new' else None)
        .sort_values(ascending=False)
        .index.tolist()
    )

    abbr_map, abbr_labels, abbr_items = abbreviate_labels(category_order)
    subset = subset[subset[protected_var].isin(category_order)].copy()
    subset['abbr_label'] = subset[protected_var].map(abbr_map)

    fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=["Favorable Interest Rate", "Favorable Action Taken"],
        horizontal_spacing=0.15
    )

    for i, dep in enumerate(['favorable_interest_rate', 'favorable_action_taken']):
        grouped = (
            subset
            .groupby(['abbr_label', dep])
            .size()
            .reset_index(name='count')
        )
        for j, val in enumerate([1, 0]):
            df = grouped[grouped[dep] == val]
            df['abbr_label'] = pd.Categorical(df['abbr_label'], categories=abbr_labels, ordered=True)
            df = df.sort_values('abbr_label')
            fig.add_trace(
                go.Bar(
                    x=df['abbr_label'],
                    y=df['count'],
                    name=f"{dep}: {val}",
                    marker_color=bar_colors[j],
                    marker_pattern_shape=pattern_shapes[j],
                    showlegend=(i == 0)
                ),
                row=1, col=i+1
            )

    fig.update_layout(
        title_text=f"{protected_label} vs Favorable Outcomes",
        barmode='group',
        height=700,
        width=1000,
        legend_title="Outcome Value"
    )
    fig.update_xaxes(tickangle=-45)

    print(f"\n===== Abbreviation Key for {protected_label} =====\n")
    for long, short in abbr_items:
        print(f"{short}: {long}")

    fig.show()


Note: you may need to restart the kernel to use updated packages.

===== Abbreviation Key for Race =====

White: White
Black or African American: Black or African American
Asian, Asian Indian: Asian, Asian Indian
Asian: Asian
Asian Indian: Asian Indian
Asian, Vietnamese: Asian, Vietnamese
Asian, Korean: Asian, Korean
Asian, Chinese: Asian, Chinese
Asian, Other Asian: Asian, Other Asian
Black or African American, White: Black or African American, White



===== Abbreviation Key for Sex =====

Male, Female: Male, Female
Female, Male: Female, Male
Female, Female: Female, Female
Male, Male: Male, Male
both,  Female: Applicant selected both male and female, Female
both,  Male: Applicant selected both male and female, Male
Male, both: Male, Applicant selected both male and female
Female, both: Female, Applicant selected both male and female
both, both: Applicant selected both male and female, Applicant selected both male and female
